In [4]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [5]:
from activ import load_data, data_normalization
import os
from activ.readfile import TrackTBIFile
import h5py
import numpy as np
from os.path import dirname, join
from IPython.display import display, Latex, Markdown
from scipy.stats import norm, sem
import scipy.optimize as spo
from activ import load_data

In [6]:
from ipywidgets import interact
import ipywidgets as widgets
from bokeh.layouts import row, widgetbox, column
from bokeh.models import Select, ColumnDataSource, ColorBar
from bokeh.palettes import Greys256, Inferno256, Plasma256, inferno, viridis, grey
from bokeh.plotting import curdoc, figure
from bokeh.transform import linear_cmap

In [7]:
nmf = load_data(uoinmf=True)
raw = load_data(uoinmf=False)

In [8]:
bm = nmf.biomarkers
bm_features = nmf.biomarker_features
oc = nmf.outcomes
oc_features = nmf.outcome_features
bm_data = raw.biomarkers
oc_data = raw.outcomes
features = np.concatenate((bm_features, oc_features)).tolist()
data = np.hstack((bm_data, oc_data))

In [9]:
from sklearn.cross_decomposition import CCA
cca = CCA(n_components=1)
bm_cca, oc_cca = cca.fit_transform(bm, oc)

In [10]:
import pandas as pd
columns = features
df = pd.DataFrame(columns=columns)

In [11]:
bm_cca = bm_cca.ravel()
oc_cca = oc_cca.ravel()

In [12]:
for idx,feature in enumerate(features):
    df[feature] = data[:,idx]

In [13]:
df = df.copy()

In [14]:
def make_plot2(df):
    xs = bm_cca
    ys = oc_cca
    
    p = figure(plot_height=1000, plot_width=1000, tools='pan,box_zoom,hover,reset')
    p.xaxis.axis_label = 'Biomarkers'
    p.yaxis.axis_label = 'Outcomes'
    
    sz = 9
    if size.value != 'No':
        size_groups = pd.Categorical(df[size.value])
        length = len(np.unique(df[size.value]))
        SIZES = list(np.linspace(6,30,length))
        if size.value == 'Age':
            size_groups = pd.Categorical(df[size.value])
            length = len(np.unique(df[size.value]))
            SIZES = list(np.linspace(6, 30, length))
        sz = [SIZES[xx] for xx in size_groups.codes]
        
    c = "#31AADE"
    if color.value != 'No':
        color_groups = pd.Categorical(df[color.value])
        length = len(np.unique(df[color.value]))
        if length > 2:
            COLORS = grey(length)
            COLORS[-1]='#dddddd'
            print(COLORS)
        else:
            COLORS = viridis(length)
        c = [COLORS[xx] for xx in color_groups.codes]
        labels = df[color.value].astype(int).values.tolist()
        mapper = linear_cmap(field_name=labels, palette=COLORS ,low=min(color_groups), high=max(color_groups))
        color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
        p.add_layout(color_bar, 'right')
        
    p.circle(x=xs, y=ys, color=c, size=sz, alpha=0.6, hover_color='white', hover_alpha=0.5)
    return p

In [15]:
def update(attr, old, new):
    layout.children[1] = make_plot2(df)
    push_notebook()

In [16]:
# plot2

from bokeh.resources import CDN
from bokeh.embed import file_html

value_col = 'GOSE_OverallScore3M'
value_sz = 'GOSE_OverallScore3M'

size = Select(title='Size', value=value_sz, options=['No']+features)
color = Select(title='Color', value=value_col, options=['No']+features)

size.on_change('value', update)
color.on_change('value', update)

controls = widgetbox([color, size], width=200)
layout = row(controls, make_plot2(df))

curdoc().add_root(layout)
h = show(layout, notebook_handle=True)
# interact(update, handle=h)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



['#000000', '#333333', '#666666', '#999999', '#cccccc', '#dddddd']
